In [2]:
import sys
import os
import inspect

sys.path.append(os.environ['GADGETRON_HOME'] + '/share/gadgetron/python')

#If Gadgetron is not installed, add the path from the gadgetron source folder
#Set GADGETRON_SOURCE to point to the location of the Gadgetron source code
#sys.path.append(os.environ['GADGETRON_SOURCE'] + '/gadgets/python/utils')
#sys.path.append(os.environ['GADGETRON_SOURCE'] + '/gadgets/python/gadgets')

import ismrmrd
import ismrmrd.xsd
import numpy as np
from ismrmrdtools import show
from gadgetron import Gadget
from gadgetron import gadget_chain_wait
from gadgetron import gadget_chain_config
from gadgetron import get_last_gadget
from tpat_snr_scale import RemOS, NoiseAdj, PCA, CoilReduce, Recon

In [9]:
print inspect.getsource(Gadget)

class Gadget(object):
    def __init__(self, next_gadget=None):
        self.next_gadget = next_gadget
        self.params = dict()
        self.results = []

    def set_parameter(self, name, value):
        self.params[name] = value

    def get_parameter(self, name):
        return self.params.get(name, None)

    def __call__(self, *args):
        self.process(args)
        return self.get_results()

    def set_next_gadget(self, gadget):
        self.next_gadget = gadget

    def process_config(self, conf):
        pass

    def process(self, header, *args):
        # do work here
        self.put_next(header,*args)

    def wait(self):
        pass
    
    def put_next(self, *args):
        if self.next_gadget is not None:
            if isinstance(self.next_gadget, Gadget):
                if len(args) == 3 and not isinstance(args[2],ismrmrd.Meta): #Data with meta data we assume
                    meta = ismrmrd.Meta()
                    meta = ismrmrd.Meta.deserialize(args[2

In [4]:
print inspect.getsource(NoiseAdj)

class NoiseAdj(Gadget):
    def __init__(self, next_gadget = None):
        Gadget.__init__(self, next_gadget)
        self.noise_data = list()
        self.noise_dmtx = None
    def process(self,acq,data,*args):
        if acq.isFlagSet(ismrmrd.ACQ_IS_NOISE_MEASUREMENT):
            self.noise_data.append((acq,data))
        else:
            if len(self.noise_data):
                profiles = len(self.noise_data)
                channels = self.noise_data[0][1].shape[0]
                samples_per_profile = self.noise_data[0][1].shape[1]
                noise = np.zeros((channels,profiles*samples_per_profile),dtype=np.complex64)
                counter = 0
                for p in self.noise_data:
                    noise[:,counter*samples_per_profile:(counter*samples_per_profile+samples_per_profile)] = p[1]
                    counter = counter + 1
                
                scale = (acq.sample_time_us/self.noise_data[0][0].sample_time_us)*0.79
                self.noise_dmtx

In [5]:
print inspect.getsource(PCA)

class PCA(Gadget):
    def __init__(self, next_gadget=None):
        Gadget.__init__(self, next_gadget) 
        self.calib_data = list()
        self.pca_mtx = None
        self.max_calib_profiles = 100
        self.samples_to_use = 16
        self.buffering = True
        
    def process(self,acq,data,*args):    
        if self.buffering:
            self.calib_data.append((acq,data))
            
            if (len(self.calib_data)>=self.max_calib_profiles or acq.isFlagSet(ismrmrd.ACQ_LAST_IN_SLICE)):
                #We are done buffering calculate pca transformation
                if self.samples_to_use < acq.number_of_samples:
                    samp_to_use = self.samples_to_use
                    
                if (len(self.calib_data) < 16):
                    samp_to_use = acq.number_of_samples
                    
                total_samples = samp_to_use*len(self.calib_data)
                channels = data.shape[0]
                
                A = np.zeros((to

In [6]:
print inspect.getsource(CoilReduce)

class CoilReduce(Gadget):
    def __init__(self, next_gadget = None):
        Gadget.__init__(self, next_gadget)
        self.coils_out = 16
        
    def process_config(self, conf):
        coils_out = self.get_parameter("coils_out")
        if (coils_out is not None):
            self.coils_out = int(coils_out)

    def process(self, acq, data, *args):
        if acq.active_channels > self.coils_out:
            data2 = data[0:self.coils_out,:]
            acq.active_channels = self.coils_out
        else:
            data2 = data
            
        self.put_next(acq,data2,*args)
        return 0



In [7]:
print inspect.getsource(RemOS)

class RemOS(Gadget):
    def process_config(self, conf):
        return

    def process(self, acq, data,*args):
        if not acq.isFlagSet(ismrmrd.ACQ_IS_NOISE_MEASUREMENT):
            ro_length = acq.number_of_samples
            padded_ro_length = (acq.number_of_samples-acq.center_sample)*2
            if padded_ro_length != ro_length: #partial fourier
                data2 = np.zeros((data.shape[0], padded_ro_length),dtype=np.complex64)
                offset = (padded_ro_length>>1)  - acq.center_sample
                data2[:,0+offset:offset+ro_length] = data
            else:
                data2 = data
    
            data2=transform.transform_kspace_to_image(data2,dim=(1,))
            data2=data2[:,(padded_ro_length>>2):(padded_ro_length>>2)+(padded_ro_length>>1)]
            data2=transform.transform_image_to_kspace(data2,dim=(1,)) * np.sqrt(float(padded_ro_length)/ro_length)
            acq.center_sample = padded_ro_length>>2
            acq.number_of_samples = data2.sh

In [8]:
print inspect.getsource(Recon)

class Recon(Gadget):
    def __init__(self, next_gadget=None):
        Gadget.__init__(self, next_gadget) 
        self.header = None
        self.enc = None
        self.acc_factor = None
        self.buffer = None
        self.samp_mask = None
        self.header_proto = None
        self.calib_buffer = list()
        self.unmix = None
        self.gmap = None
        self.calib_frames = 0
        self.method = 'grappa'
    
    def process_config(self, cfg):
        self.header = ismrmrd.xsd.CreateFromDocument(cfg)
        self.enc = self.header.encoding[0]

        #Parallel imaging factor
        self.acc_factor = self.enc.parallelImaging.accelerationFactor.kspace_encoding_step_1
        
        reps = self.enc.encodingLimits.repetition.maximum+1
        phs = self.enc.encodingLimits.phase.maximum+1
        if reps > phs:
            self.calib_frames = reps
        else:
            self.calib_frames = phs
            
        if self.calib_frames < self.acc_factor:
            